In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import os

import numpy as np
import pandas as pd

import re
import string
# spacy for lemmatization
import spacy

# NLTK Stop words
#import nltk
#from nltk.corpus import stopwords
#from nltk import TweetTokenizer
#from nltk.stem import WordNetLemmatizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models import Word2Vec
# Plots
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','good','be','class','method','function','return','code','fix','add'])

#f = open('/Users/admin/Documents/Winter2020/RevicedRelease Eng/dataset10/stopword.txt', 'r')
#x = f.readlines()
#remove_words = []
#for xx in x:
#    remove_words.append(xx.replace("\n",''))
#f.close()

In [3]:
# Import Dataset
df = pd.read_csv('/Users/admin/Documents/commit-message/cleaned/Ccommitmessages.csv')
#print(df.Text.unique())
df.head()


,System,Text
0,ztd-jni,Fix all warnings Also add tests to validate th...
1,ztd-jni,Zstd v0.4.4/Fix the 32bit compatibility We nee...
2,ztd-jni,Zstd 0.3 add compression using HC variant/
3,ztd-jni,Concurrency fix/Zstd-0.4.1 and performance tes...
4,ztd-jni,Zstd 0.3 add compression using HC variant/


In [4]:
# Convert to list
data = df.Text.values.tolist()
# Remove new line characters
#for line in data:
    #print(line)
#data = [re.sub('\s+',' ', str(sent)) for sent in data]
#data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
#data = [re.sub('<pre>(.*?)</pre>', '', sent) for sent in data]
#data = [re.sub('<code>(.*?)</code>', '', sent) for sent in data]
#data = [re.sub('<a(.*?)</a>', ' ', sent) for sent in data]
#data = [re.sub('<p>', ' ', sent) for sent in data]
#data = [re.sub('</p>', ' ', sent) for sent in data]
#data = [re.sub('<li>', ' ', sent) for sent in data]
#data = [re.sub('</li>', ' ', sent) for sent in data]
#data = [re.sub('<ol>', ' ', sent) for sent in data]
#data = [re.sub('</ol>', ' ', sent) for sent in data]
#data = [re.sub('<em>', ' ', sent) for sent in data]
#data = [re.sub('</em>', ' ', sent) for sent in data]
#data = [re.sub('<blockquote>', ' ', sent) for sent in data]
#data = [re.sub('</blockquote>', ' ', sent) for sent in data]
#data = [re.sub('<h1>', ' ', sent) for sent in data]
#data = [re.sub('</h1>', ' ', sent) for sent in data]
#data = [re.sub('<h2>', ' ', sent) for sent in data]
#data = [re.sub('</h2>', ' ', sent) for sent in data]
#data = [re.sub('\'', '', sent) for sent in data]

In [5]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures



def get_bigrams(myString):
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(myString)
    stemmer = PorterStemmer()
    bigram_finder = BigramCollocationFinder.from_words(tokens)
    bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)

    for bigram_tuple in bigrams:
        x = "%s_%s" % bigram_tuple
        tokens.append(x)

    result = [' '.join([stemmer.stem(w).lower() for w in x.split()]) for x in tokens if x.lower() not in stop_words]
    return result

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
data_words_nostops = remove_stopwords(data_words)

In [8]:
sentences = []
bigram = Phrases()
for sentence in data_words_nostops:
    sentences.append(sentence)
    bigram.add_vocab([sentence])
#print(list(bigram[sentences])[:5])

In [9]:

#for data in 
# Form Bigrams
data_words_bigrams = list(bigram[sentences])

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized[:1])

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]]


In [11]:
# Validate the term frequency
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('also', 1),
  ('always', 1),
  ('bit', 2),
  ('bytestream', 1),
  ('cast', 2),
  ('compatibility', 1),
  ('compiler_warning', 1),
  ('double', 1),
  ('else', 1),
  ('equal', 1),
  ('jlong', 1),
  ('need', 1),
  ('order', 1),
  ('pointer', 2),
  ('produce', 1),
  ('rebase', 1),
  ('size', 1),
  ('test', 1),
  ('top', 1),
  ('validate', 1),
  ('warning', 1),
  ('zstd', 1)]]

In [12]:
output_path = '/Users/admin/Documents/commit-message/Topics/20/'
num_topics = 16
step_size = 4
topic_index = 3

In [13]:
# Build LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           per_word_topics=True)

In [14]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.226295097933248

Coherence Score:  0.30768447626117534


In [15]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=num_topics)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [16]:
df_dominant_topic.to_csv(r'{}Topic_1.csv'.format(output_path))
# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,15.0,0.7039,"test, support, change, version, fix, restructu...",Fix all warnings Also add tests to validate th...
1,1,15.0,0.9506,"test, support, change, version, fix, restructu...",Zstd v0.4.4/Fix the 32bit compatibility We nee...
2,2,5.0,0.7645,"test, summary, test_plan, make, throwing_excep...",Zstd 0.3 add compression using HC variant/
3,3,0.0,0.7201,"summary, test, test_plan, differential_revisio...",Concurrency fix/Zstd-0.4.1 and performance tes...
4,4,5.0,0.7645,"test, summary, test_plan, make, throwing_excep...",Zstd 0.3 add compression using HC variant/
5,5,3.0,0.4983,"test, support, also, summary, use, change, cor...",Use JNI_ABORT on releasing the source buffer c...
6,6,3.0,0.4988,"test, support, also, summary, use, change, cor...",Use JNI_ABORT on releasing the source buffer c...
7,7,13.0,0.8791,"platform, session, make, sessionsnapshot, acti...",Add support for InputStream.read() FilterInput...
8,8,13.0,0.6873,"platform, session, make, sessionsnapshot, acti...",Move to the new streaming API/Zstd-0.7.4/
9,9,15.0,0.7285,"test, support, change, version, fix, restructu...",Finish the move to the new Zstd streaming API ...


In [17]:
from gensim.corpora import Dictionary
from pprint import pprint
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/Users/admin/Documents/mallet-2.0-2.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(14,
  [('support', 0.04161588020198503),
   ('key', 0.039874629984328745),
   ('implementation', 0.03290962911370364),
   ('instance', 0.023506877938359743),
   ('implement', 0.02211387776423472),
   ('provide', 0.021069127633640956),
   ('cipher_suite', 0.019502002437750306),
   ('enable', 0.018805502350687793),
   ('work', 0.014104126763015845),
   ('list', 0.013407626675953334)]),
 (0,
  [('compaction', 0.04824955116696589),
   ('level', 0.04757630161579892),
   ('file', 0.04207809694793537),
   ('summary', 0.03949730700179533),
   ('test_plan', 0.030745062836624776),
   ('write', 0.022778276481149013),
   ('option', 0.020758527827648116),
   ('run', 0.020197486535008975),
   ('differential_revision', 0.01671903052064632),
   ('number', 0.016494614003590664)]),
 (9,
  [('commit', 0.06296769483482387),
   ('call', 0.03814564701587881),
   ('read', 0.030297499543712355),
   ('merge', 0.02701222850885198),
   ('change', 0.02646468333637525),
   ('buffer', 0.023909472531483848),
   ('

In [18]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=step_size, step=step_size):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [19]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=step_size, limit=40, step=step_size)

In [20]:
# Show graph
limit=40; start=step_size; step=step_size;
x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
#plt.show()

In [21]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 4  has Coherence Value of 0.3979
Num Topics = 8  has Coherence Value of 0.3975
Num Topics = 12  has Coherence Value of 0.3818
Num Topics = 16  has Coherence Value of 0.4353
Num Topics = 20  has Coherence Value of 0.4457
Num Topics = 24  has Coherence Value of 0.4312
Num Topics = 28  has Coherence Value of 0.4415
Num Topics = 32  has Coherence Value of 0.4354
Num Topics = 36  has Coherence Value of 0.443


In [22]:
# Select the model and print the topics
optimal_model = model_list[topic_index]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.033*"test" + 0.031*"summary" + 0.029*"test_plan" + 0.026*"file" + '
  '0.019*"differential_revision" + 0.016*"unit" + 0.014*"reviewer" + '
  '0.014*"add" + 0.013*"case" + 0.012*"delete"'),
 (1,
  '0.048*"file" + 0.036*"summary" + 0.027*"test_plan" + '
  '0.025*"differential_revision" + 0.024*"option" + 0.024*"test" + '
  '0.019*"delete" + 0.019*"compaction" + 0.018*"time" + 0.014*"unit_test"'),
 (2,
  '0.045*"commit" + 0.038*"call" + 0.029*"enable" + 0.022*"merge" + '
  '0.019*"cipher_suite" + 0.017*"buffer" + 0.016*"instance" + 0.015*"key" + '
  '0.012*"array" + 0.011*"check"'),
 (3,
  '0.036*"type" + 0.028*"case" + 0.016*"load" + 0.015*"order" + '
  '0.014*"default" + 0.014*"main_file" + 0.014*"parameter" + 0.013*"cacert" + '
  '0.012*"create" + 0.011*"make"'),
 (4,
  '0.043*"type" + 0.026*"fix" + 0.025*"update" + 0.023*"support" + '
  '0.020*"long" + 0.018*"exception" + 0.016*"remove" + 0.015*"field" + '
  '0.012*"set" + 0.012*"java"'),
 (5,
  '0.036*"realm" + 0.029*"check

In [23]:
def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=20)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

df_dominant_topic.to_csv(r'{}Topic_2.csv'.format(output_path))
# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9.0,0.1207,"issue, pointer, native, error, call, object, p...",Fix all warnings Also add tests to validate th...
1,1,9.0,0.1113,"issue, pointer, native, error, call, object, p...",Zstd v0.4.4/Fix the 32bit compatibility We nee...
2,2,11.0,0.0793,"table, test_plan, summary, reviewer, log, user...",Zstd 0.3 add compression using HC variant/
3,3,7.0,0.1038,"fix, find, check, update, issue, request_resol...",Concurrency fix/Zstd-0.4.1 and performance tes...
4,4,11.0,0.0793,"table, test_plan, summary, reviewer, log, user...",Zstd 0.3 add compression using HC variant/
5,5,2.0,0.0763,"commit, call, enable, merge, cipher_suite, buf...",Use JNI_ABORT on releasing the source buffer c...
6,6,0.0,0.0743,"test, summary, test_plan, file, differential_r...",Use JNI_ABORT on releasing the source buffer c...
7,7,7.0,0.1113,"fix, find, check, update, issue, request_resol...",Add support for InputStream.read() FilterInput...
8,8,7.0,0.0793,"fix, find, check, update, issue, request_resol...",Move to the new streaming API/Zstd-0.7.4/
9,9,7.0,0.1178,"fix, find, check, update, issue, request_resol...",Finish the move to the new Zstd streaming API ...


In [24]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]


sent_topics_sorteddf_mallet.to_csv(r'{}Topic_3.csv'.format(output_path))
# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.6479,"test, summary, test_plan, file, differential_r...",Fix the string format issue Summary: mac and o...
1,1.0,0.7450,"file, summary, test_plan, differential_revisio...",Universal Compaction should keep DeleteMarkers...
2,2.0,0.8779,"commit, call, enable, merge, cipher_suite, buf...",Fix OpenSSLSocketImpl.getPort when SNI is used...
3,3.0,0.7049,"type, case, load, order, default, main_file, p...",Thanks to Ken Olson reporting MSVC2010 complai...
4,4.0,0.5451,"type, fix, update, support, long, exception, r...",Remove Java OpenSSL name mapping. (#227) As of...


In [25]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

df_dominant_topics.to_csv(r'{}Topic_4.csv'.format(output_path))
# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,9.0,"issue, pointer, native, error, call, object, p...",150.0,0.0689
1.0,9.0,"issue, pointer, native, error, call, object, p...",57.0,0.0262
2.0,11.0,"table, test_plan, summary, reviewer, log, user...",106.0,0.0487
3.0,7.0,"fix, find, check, update, issue, request_resol...",109.0,0.0500
4.0,11.0,"table, test_plan, summary, reviewer, log, user...",249.0,0.1143
...,...,...,...,...
2173.0,0.0,"test, summary, test_plan, file, differential_r...",NaN,NaN
2174.0,15.0,"test, change, support, harmony_xnet, remove, m...",NaN,NaN
2175.0,13.0,"support, implementation, implement, work, key,...",NaN,NaN
2176.0,0.0,"test, summary, test_plan, file, differential_r...",NaN,NaN
